# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

- *population* — the global population.
- *population_growth* — the annual population growth rate, as a percentage.
- *area* — the total land and water area.

We'll use SQL in Jupyter Notebook to analyze data from this database. Let's connect our notebook to the database:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Let´s get information about the tables in our database.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We can see the first five rows in the **facts** table: 

In [3]:
%%sql

SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


We have a lot of information about different countries like *area* or *population*. Let's start by calculating some summary statistics and look for any outlier countries.

In [4]:
%%sql
SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [5]:
%%sql

SELECT name, population
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0


In [6]:
%%sql

SELECT name, population
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts);

 * sqlite:///factbook.db
Done.


name,population
World,7256490011


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

In [7]:
%%sql
SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts
 WHERE name <> 'Antarctica' AND name <> 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


In [8]:
%%sql

SELECT AVG(population),
       AVG(area)
  FROM facts;

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have the following:

- Above-average values for population.
- Below-average values for area.

In [9]:
%%sql

SELECT name,
       population,
       area
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                    ) AND
       area < (SELECT AVG(area)
                 FROM facts
              );

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


There are a few countries whit high density population. We also have information about the countries' land and water area:
- *area_land* — the country's land area in square kilometers.
- *area_water* — the country's waterarea in square kilometers.

We can calculate water to land ratio, dividing water area with land area. Is there any country that has more water than land? Let's find out:

In [10]:
%%sql

SELECT name, 
       CAST(area_water AS Float) / CAST(area_land AS Float) AS water_to_land
  FROM facts
 ORDER BY water_to_land DESC
 LIMIT 20;

 * sqlite:///factbook.db
Done.


name,water_to_land
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


When land area is bigger than water area, ratio is below 1. Therefore, we can see there are two countries with bigger water area than land area: [British Indian Ocean Territory](https://en.wikipedia.org/wiki/British_Indian_Ocean_Territory) and [Virgin Islands](https://en.wikipedia.org/wiki/Virgin_Islands). Searching on the web about this countries, we can see data is right.
Database also provides information about birth rates and death rates, here's the information about the columns:
- *birth_rate* — the country's birth rate, or the number of births per year per 1,000 people.
- *death_rate* — the country's death rate, or the number of death per year per 1,000 people.

Are there any countries with higher death rates than birth rates?

In [11]:
%%sql

SELECT name,
       death_rate,
       birth_rate
  FROM facts
 WHERE death_rate > birth_rate

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16


Having a death rate higher than birth rate would lead to a decrease in population.